# Build a Docker image for an MCP server 
This tutorial builds a Docker image for an MCP server using Podman, specifically tailored for deploying on Anyscale with Ray Serve. It complements the official Anthropic tutorial for [Building the MCP weather server](https://modelcontextprotocol.io/quickstart/server), which doesn't include Docker image-building instructions.

Docker simplifies the process of building and distributing MCP servers by packaging them into standardized, portable containers. This eliminates issues related to dependencies and environment configuration. It also enables streamlined cloud development—enhancing testing, security, and cross-platform deployment for agent-focused tools.

Unfortunately, you can't use the [Anyscale Docker image build farm](https://docs.anyscale.com/configuration/dependency-management/dependency-container-images/) to build this image, as it doesn't support the Docker `COPY` command from local storage.

Therefore, this tutorial shows you how to build the MCP Docker image directly from the Anyscale workspace. Alternatively, you can also [build it from your local machine](https://docs.anyscale.com/container-image/build-image-tutorial). 

See https://hub.docker.com/catalogs/mcp for the Docker images collection for MCP servers. 

## Prerequisites and folder layout

Ensure you have Podman installed:
```bash
podman --version
```

See the `build-mcp-docker-image` folder that contains the following files:

```text
build-mcp-docker-image/
├── Dockerfile     
├── requirements.txt
└── weather.py      
```

## 1. Dockerfile for a weather MCP server

This Dockerfile creates a lightweight image based on `python3.10` for running a weather MCP server. It installs system dependencies and the [`uv`](https://github.com/astral-sh/uv) CLI tool for efficient package and application management. 

The Dockerfile sets the working directory to `/app`, installs Python packages from `requirements.txt` using uv, copies the full project into the container, and finally runs the `weather.py` server script using `uv`.


```python
# Use Python 3.10 base image.
FROM python:3.10-slim

# Install system dependencies.
RUN apt-get update && \
    apt-get install -y curl ca-certificates && \
    rm -rf /var/lib/apt/lists/*

# Install the 'uv' CLI.
RUN curl -LsSf https://astral.sh/uv/install.sh | sh

# Make sure 'uv' is on PATH.
ENV PATH="/root/.local/bin:${PATH}"

# Set the working directory.
WORKDIR /app

# Copy and install only requirements first (caching).
COPY requirements.txt .
RUN uv pip install --system -r requirements.txt

# Now copy everything from the current directory into /app.
COPY . .

# Run the server.
CMD ["uv", "run", "weather.py"]
```


## 2.  Build a Docker image with Podman

Navigate to the project directory:

```bash
cd build-mcp-docker-image
```

Run the following Podman command to build the Docker image, ensuring you use the --events-backend=file flag to prevent build errors on Anyscale:
```bash
podman build \
  --events-backend=file \
  -t weather-mcp:latest .
```

**Note**: 

Omitting the `--events-backend=file` flag may result in the following error:
```text
ERRO[0000] unable to write build event: "write unixgram @11c5a->/run/systemd/journal/socket: sendmsg: no such file or directory"
```




## 3. Push the Docker image to registry

Push the built image to your own container registry. You should replace `your-dockerhub-username/weather-mcp` with your actual image name:

```bash
podman tag weather-mcp:latest your-dockerhub-username/weather-mcp:latest

podman push \
  --events-backend=file \
  your-dockerhub-username/weather-mcp:latest
```

Make sure you're logged into your Docker Hub account or container registry before pushing:
```bash
podman login docker.io
```


## Next steps
You have successfully built and pushed a Docker image for your MCP server, which is ready to deploy.

Once you push the image, you can deploy the MCP server on Anyscale using Ray Serve. 

Follow the next two tutorials for single and multiple MCP server deployments using Ray Serve.